In [1]:
import numpy as np
import csv
import itertools
import pandas as pd
import sys
import tempfile
import tqdm
import glob
import random

from sklearn.model_selection import train_test_split


def load_X(X_path):
    file = open(X_path, 'r')
    X_ = np.array(
        [elem for elem in [
            row.split(',')[:] for row in file
        ]],
        dtype=np.float32
    )
    X_=X_[:,:-1]
    print("X_.shape is ",str(X_.shape))
    file.close()
    return X_
def load_Y(Y_path):
    file = open(Y_path, 'r')
    Y_ = np.array(
        [elem for elem in [
            row.split(',')[:] for row in file
        ]],
        dtype=np.float32
    )
    Y_=Y_[:,-1]
    print("Y_shape is" ,str(Y_.shape))
    file.close()
    return Y_
def load_filepath(path):
    file = open(path, 'r')
    fp_=[]
    for row in file:
      temp_fp_=row.split(',')[-1]
      fp_.append(temp_fp_)
    print("Length of file_path is: ",len(fp_))
    file.close()
    return fp_

# Swapping upper parts and lower parts
max_samples=48800
df=pd.read_csv('mixed/trainmixed.csv', header=None)

augmented_df_list=[]
with open('mixed/fullswaptrainmixed.csv', 'w', newline='') as sample_chosen_out_csv_file:
    sample_chosen_out_writer = csv.writer(sample_chosen_out_csv_file)
    for pose in range(0,9): 
        num_sample=0
        print(pose) 
        rows_pose_mask=df.iloc[:,-1]==pose
        rows_pose=df[rows_pose_mask]
        # Shuffle the rows
        rows_pose = rows_pose.sample(frac=1,random_state=123).reset_index(drop=True)
        print(f"class {pose} originally have {len(rows_pose)} samples")
        for row_pose_index_i in range (0,len(rows_pose)):
            sample_chosen_out_writer.writerow(list(rows_pose.iloc[row_pose_index_i]))
            num_sample+=1
        for row_pose_index_i in range (0,len(rows_pose)-1):
            if num_sample >=max_samples:
                break
            for row_pose_index_j in range (row_pose_index_i+1,len(rows_pose)):
                # trao lan 1
                new_keypoint=[0.0]*33
                new_keypoint[0:16]=list(rows_pose.iloc[row_pose_index_i,:16])
                new_keypoint[16:]=list(rows_pose.iloc[row_pose_index_j,16:])
                # tinh lai toa do spine
                new_keypoint[7*2]=(new_keypoint[0*2]+new_keypoint[8*2])/2.0
                new_keypoint[7*2+1]=(new_keypoint[0*2+1]+new_keypoint[8*2+1])/2.0
                sample_chosen_out_writer.writerow(new_keypoint)
                
                # trao lan 2
                new_keypoint=[0.0]*33
                new_keypoint[0:16]=list(rows_pose.iloc[row_pose_index_j,:16])
                new_keypoint[16:]=list(rows_pose.iloc[row_pose_index_i,16:])
                # tinh lai toa do spine
                new_keypoint[7*2]=(new_keypoint[0*2]+new_keypoint[8*2])/2.0
                new_keypoint[7*2+1]=(new_keypoint[0*2+1]+new_keypoint[8*2+1])/2.0
                sample_chosen_out_writer.writerow(new_keypoint)
                

                num_sample+=2
                if num_sample >=max_samples:
                    break
        print(f"class {pose} now have {num_sample}/{max_samples} samples. It is done")

FileNotFoundError: [Errno 2] No such file or directory: 'trainmixed.csv'

Another type of swap : swapping left->right hand, left-right leg

In [1]:
import numpy as np
import csv
import pandas as pd

def swap_coordinates(row):
    # print("haha")
    # print(len(row))
    # Swapping coordinates as specified
    swapped_row=[]
    swapped_index = [0,4,5,6,1,2,3,7,8,9,13,14,15,10,11,12]
    for idx in swapped_index:
        swapped_row.append(row[2*idx])
        swapped_row.append(row[2*idx+1])
    swapped_row.append(row[32])
    return swapped_row

def swap_and_append_rows(input_csv, output_csv):
    original_count = 0
    new_count = 0

    df = pd.read_csv(input_csv, header=None)

    with open(output_csv, 'w', newline='') as output_file:
        csv_writer = csv.writer(output_file)
        for index, row in df.iterrows():
            # Original row
            csv_writer.writerow(row)
            original_count += 1

            # Swapped row
            swapped_row = swap_coordinates(row)
            csv_writer.writerow(swapped_row)
            new_count += 1

    return original_count, new_count

if __name__ == "__main__":
    input_csv = '/ske/Kinetic-GAN/output/gan_sample/gan_sample.csv'
    output_csv = '/ske/Kinetic-GAN/output/gan_sample/swap_gan_sample.csv'
    original_count, new_count = swap_and_append_rows(input_csv, output_csv)

    print("Original CSV file length:", original_count)
    print("New CSV file length after swapping coordinates:", new_count)

Original CSV file length: 889665
New CSV file length after swapping coordinates: 889665


In [2]:
import csv

def delete_last_row(input_csv, output_csv):
    with open(input_csv, 'r') as file:
        lines = list(csv.reader(file))
    
    # Remove the last row
    lines = lines[:-1]

    with open(output_csv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(lines)

if __name__ == "__main__":
    input_csv = 'sideganfullswaptraingt16.csv'  # Specify the input CSV file path
    output_csv = 'sideganfullswaptraingt16.csv'  # Specify the output CSV file path after deleting the last row

    delete_last_row(input_csv, output_csv)
    print("Last row deleted successfully.")

Last row deleted successfully.


Here is another type of augment : flip horizontally

In [1]:

# Read input CSV file
input_file = 'fullswaptraingt16.csv'
output_file = 'flipfullswaptraingt16.csv'
import csv

# Function to flip coordinates horizontally
def flip_coordinates(row):
    flipped_row = row
    for i in range(0, len(flipped_row) - 1, 2):
        flipped_row[i] = str(120.0 - float(flipped_row[i]))
    return flipped_row



with open(input_file, 'r') as f_in, open(output_file, 'w', newline='') as f_out:
    reader = csv.reader(f_in)
    writer = csv.writer(f_out)

    for row in reader:
        class_id = float(row[-1])
        if class_id in [0, 1, 2]:
            # Flip coordinates horizontally and write both original and flipped rows
            flipped_row = flip_coordinates(row[:-1]) + [str(class_id)]  # Keep class_id
            writer.writerow(row)
            writer.writerow(flipped_row)
        elif class_id in [3, 4, 5]:
            # Write original row and create new row with flipped coordinates and updated class_id
            writer.writerow(row)
            flipped_row = flip_coordinates(row[:-1]) + [str(class_id + 3)]  # Update class_id
            writer.writerow(flipped_row)
        elif class_id in [6, 7, 8]:
            # Write original row and create new row with flipped coordinates and updated class_id
            writer.writerow(row)
            flipped_row = flip_coordinates(row[:-1]) + [str(class_id - 3)]  # Update class_id
            writer.writerow(flipped_row)

print("Output CSV file has been created successfully.")


ValueError: invalid literal for int() with base 10: '0.0'